In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
data_path = '../../Data/data_min_R/' # Charlie's local.

ssps = ['ssp1','ssp2','ssp3','ssp4','ssp5']

# Calc global %LS_2050 
For each SSP, from Bodirsky's data and Jones' populations.

## Load data

In [11]:
## Load countries
countries = pd.read_csv(data_path + 'inputs/Base/' + 'country_ids.csv')
countries_names = pd.read_csv(data_path + 'inputs/Base/' + 'country_names.csv',encoding='latin-1')
countries = countries[['country_id','pixel_id']].merge(countries_names[['base_id','ISO3','name', 'sovereignt']],right_on='base_id',left_on='country_id',how='right')
countries = countries.set_index('pixel_id')

In [7]:
## Load population

def import_old_pop_data(scenario):
    """Takes scenario in format '2000', 'ssp5'"""
    
    countries = pd.read_csv(data_path + 'inputs/Base/' + 'country_ids.csv')
    countries_names = pd.read_csv(data_path + 'inputs/Base/' + 'country_names.csv',encoding='latin-1')
    countries = countries[['country_id','pixel_id']].merge(countries_names[['base_id','ISO3','name', 'sovereignt']],right_on='base_id',left_on='country_id',how='right')
    countries = countries.set_index('pixel_id')
    
    pop_dir = data_path + 'inputs/population'
    pop = pd.read_csv(os.path.join(pop_dir,'pop_'+ scenario + '.csv'))
    pop = pop.merge(countries, right_on='pixel_id', left_on='pixel_id')
    pop_bycountry = pop.groupby('name')['gpw_population'].sum()
    
    
    """# Combine the Cyprus and Northern Cyprus as we only have one in our future data
    pop_bycountry.loc['Cyprus'] += pop_bycountry['Northern Cyprus']
    pop_bycountry = pop_bycountry.drop('Northern Cyprus')"""
    return pop_bycountry

In [96]:
population = pd.DataFrame()
for ssp in ssps+['2000']:
    population['pop_'+ssp]= import_old_pop_data(ssp)

In [97]:
population = population.merge(countries[['name','ISO3']].set_index('name').drop_duplicates(),on='name')
population = population.set_index('ISO3')
population

,pop_ssp1,pop_ssp2,pop_ssp3,pop_ssp4,pop_ssp5,pop_2000
ISO3,,,,,,
AFG,52765426,66994422,75630510,76377078,52105184,1.975393e+07
WSB,16614,16614,16614,16614,33227,1.413851e+04
ALD,15540,15540,15540,15540,15540,1.414138e+04
ALB,2794859,2797509,2797509,2794859,2794859,3.012125e+06
DZA,35193987,35363778,62463440,35197306,35193979,3.046609e+07
...,...,...,...,...,...,...
WLF,2180,2180,2180,2180,2180,2.515850e+03
SAH,2007,2480,2629,2483,2086,1.964445e+03
YEM,45584457,53381988,63593696,65905907,45312856,1.748859e+07


In [4]:
## Load %LS

diets_path = os.path.join(data_path,'inputs','Diet','pctLS_SSPs_Bodirsky.csv')
diets = pd.read_csv(diets_path, sep=';')[['Scenario', 'Region', '2050', '2000']].set_index('Scenario')
diets

In [ ]:
### Merge all 



In [26]:
c = countriepopulatios[['name','ISO3']].set_index('name').drop_duplicates()#.unique()#.#,on='name',how='left')

In [13]:
diets

,Region,2050,2000
Scenario,,,
ssp1,AFG,0.1854,0.0810
ssp1,AGO,0.1531,0.0721
ssp1,ALB,0.3411,0.2535
ssp1,ARE,0.1458,0.2078
ssp1,ARG,0.3329,0.3070
...,...,...,...
ssp5,WSM,0.2765,0.2543
ssp5,YEM,0.1468,0.0655
ssp5,ZAF,0.2026,0.1214


## Global %LS_ssp = $\sum_{Countries}$ ($\%LS_{Country}$ * $Pop_{Country}$) / $Pop_{Total}$

Average of %LS, weighted by pop

In [90]:
ssp = 'ssp1'
numerator = 0
denominator = 0
country_list = diets['Region'].unique()

for ssp in ssps:
    for country in country_list:
        # Retrieve data from diets and population
        try:
            pctLS_country = diets[diets['Region']==country].at[ssp,'2050']
            pop_country = population.at[country,'pop_'+ssp]

            LS_country = pctLS_country * pop_country
            numerator+=LS_country
            denominator+= pop_country
        except:
            print('Skipped '+country+' (not in population)')

    global_cal_suff = numerator/denominator
    
    print('Global Caloric Sufficiency '+ssp+': {}'.format(global_cal_suff))

Skipped KSV (not in population)
Skipped MAF (not in population)
Skipped MDV (not in population)
Skipped MHL (not in population)
Skipped TUV (not in population)
Global Caloric Sufficiency ssp1: 0.23728686354880824
Skipped KSV (not in population)
Skipped MAF (not in population)
Skipped MDV (not in population)
Skipped MHL (not in population)
Skipped TUV (not in population)
Global Caloric Sufficiency ssp2: 0.22905550297316568
Skipped KSV (not in population)
Skipped MAF (not in population)
Skipped MDV (not in population)
Skipped MHL (not in population)
Skipped TUV (not in population)
Global Caloric Sufficiency ssp3: 0.20972663721021714
Skipped KSV (not in population)
Skipped MAF (not in population)
Skipped MDV (not in population)
Skipped MHL (not in population)
Skipped TUV (not in population)
Global Caloric Sufficiency ssp4: 0.20811659841733302
Skipped KSV (not in population)
Skipped MAF (not in population)
Skipped MDV (not in population)
Skipped MHL (not in population)
Skipped TUV (not in 

In [94]:
global_pctLS = {'ssp1':0.23728686354880824,
               'ssp2':0.22905550297316568,
               'ssp3':0.20972663721021714,
               'ssp4':0.20811659841733302,
               'ssp5':0.21198181981961245}

In [93]:
population.sum() # Phew! Global pops are good :)

pop_ssp1    7949926714
pop_ssp2    8210591474
pop_ssp3    9929316985
pop_ssp4    8722972624
pop_ssp5    8248102437
dtype: int64

## Global pct_LS_2000

In [98]:
ssp = 'ssp1'
numerator = 0
denominator = 0
country_list = diets['Region'].unique()

for country in country_list:
        # Retrieve data from diets and population
    try:
        pctLS_country = diets[diets['Region']==country].at[ssp,'2000']
        pop_country = population.at[country,'pop_'+ssp]

        LS_country = pctLS_country * pop_country
        numerator+=LS_country
        denominator+= pop_country
    except:
        print('Skipped '+country+' (not in population)')

global_cal_suff = numerator/denominator

print('Global Caloric Sufficiency 2000 : {}'.format(global_cal_suff))

Skipped KSV (not in population)
Skipped MAF (not in population)
Skipped MDV (not in population)
Skipped MHL (not in population)
Skipped TUV (not in population)
Global Caloric Sufficiency 2000 : 0.13274634152807852


## Understand countries variability ( Where highest, in HIC, everywhere??)
Map countries %LS (Bodirsky data) for all SSP and 2000

To do !

# Global Caloric Sufficiency

In [ ]:
global_pctLS = {'ssp1':0.23728686354880824,
               'ssp2':0.22905550297316568,
               'ssp3':0.20972663721021714,
               'ssp4':0.20811659841733302,
               'ssp5':0.21198181981961245}

In [ ]:
To do ! 